### Extração de dados youtube- visualizacão de videos

**Bibliotecas:**

* pandas
* numpy
* BeautifulSoup
* matplotlib

In [13]:
 pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


**Imports de bliotecas para utilização:**

In [14]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import codecs
from datetime import datetime

***Local do arquivo html***

In [15]:
local_html = '/home/mateus/Downloads/youtube/histórico/histórico-de-visualização.html'

**Instância de objeto da biblioteca BeautifulSoup**

In [16]:
def cria_objeto_soup(html_doc):
    fp = codecs.open(html_doc, 'r') 
    sp = BeautifulSoup(fp)
    return sp

In [17]:
def coleta_divs(nome_div,obj):
    mydivs = obj.findAll("div", {"class": nome_div})
    return  mydivs

In [18]:
def cria_dicionario():
    informacoe_visualizacao = {
                                        'canal':[],
                                        'url_canal':[],
                                        'nome_video':[],
                                        'url_video':[],
                                        'data_visualizacao':[]
    }
    return informacoe_visualizacao

In [19]:
nome_divs = 'content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1'
soup = cria_objeto_soup(local_html)
divs = coleta_divs(nome_divs,soup)


In [20]:
dados = cria_dicionario()
for dado in divs:
    href_video = dado.a

    dados_novo_tag_a = dado.find_all('a')  
    date = str(dado).split('<br/>')
    tm_date = len(date)
    date = date[tm_date-1].split('</div>')
    txt = dado.text
    tm = len('Watched\xa0')
    txt = txt[tm:]
    
    
    try: 
       dados['canal'].append(dados_novo_tag_a[1].text)
    except:
       dados['canal'].append(np.NaN)
    
    try: 
       dados['url_canal'].append(dados_novo_tag_a[1]['href'])
    except:
       dados['url_canal'].append(np.NaN) 
    
    try:
       dados['nome_video'].append(txt) 
    except:
       dados['nome_video'].append(np.NaN)
    
    try:
       dados['url_video'].append(href_video['href'])
    except:
       dados['url_video'].append(np.NaN)
    
    try:  
       dados['data_visualizacao'].append(str(date[0]))
    except:
       dados['data_visualizacao'].append(np.NaN)
  

In [36]:
visualizao_youtube = pd.DataFrame(dados)

In [37]:
visualizao_youtube.head(10)

,canal,url_canal,nome_video,url_video,data_visualizacao
0,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,EDUARDO BOLSONARO E ANDRÉ MARINHO TRETARAM FEI...,https://www.youtube.com/watch?v=PD9Lk1YleWk,25 de jul. de 2021 17:55:05 BRT
1,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,GABRIEL MONTEIRO NO PÂNICO - MELHORES MOMENTOS...,https://www.youtube.com/watch?v=fOB0GYmAz7U,25 de jul. de 2021 17:55:00 BRT
2,Gabriel Monteiro,https://www.youtube.com/channel/UCYCMK_puWK2V5...,"VEIO ME EXPULSAR DO HOSPITAL, MAS FOI FLAGRADO...",https://www.youtube.com/watch?v=RpfoCFRGHbo,25 de jul. de 2021 17:54:57 BRT
3,Programação Dinâmica,https://www.youtube.com/channel/UC70mr11REaCqg...,RASPAGEM de DADOS com PYTHON usando BeautifulS...,https://www.youtube.com/watch?v=kqvWOcPog4s,25 de jul. de 2021 02:09:53 BRT
4,Jonny Kalambay,https://www.youtube.com/channel/UCUm0chMKj4MA7...,Como Utilizar la API de SpotifyJonny Kalambay2...,https://www.youtube.com/watch?v=PfL_v6A4fsI,23 de jul. de 2021 22:36:44 BRT
5,Dev Aprender,https://www.youtube.com/channel/UCm63tB8wsKOVv...,Criei uma NOVA FUNCIONALIDADE no Spotify com P...,https://www.youtube.com/watch?v=AJcSX_IfDp8,23 de jul. de 2021 22:36:42 BRT
6,Otávio Miranda,https://www.youtube.com/channel/UCORZcu08VQiRC...,Consumindo uma API Rest usando requests em Pyt...,https://www.youtube.com/watch?v=Qd8JT0bnJGs,23 de jul. de 2021 22:35:30 BRT
7,MotechApp,https://www.youtube.com/channel/UCtuaigKZF3okQ...,"How To Copy, Append And Replace In Python Pand...",https://www.youtube.com/watch?v=Tp7SH95RKl4,20 de jul. de 2021 23:46:01 BRT
8,Geração Analítica,https://www.youtube.com/channel/UChLeZUQMhvyBa...,Unindo Dataframes - Merge | Concat | Join Data...,https://www.youtube.com/watch?v=ubMG7VjPmhs,20 de jul. de 2021 23:31:04 BRT
9,Data School,https://www.youtube.com/channel/UCnVzApLJE2ljP...,How do I merge DataFrames in pandas?Data Schoo...,https://www.youtube.com/watch?v=iYWKfUOtGaw,20 de jul. de 2021 23:30:53 BRT


In [38]:
visualizao_youtube.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   canal              825 non-null    object
 1   url_canal          825 non-null    object
 2   nome_video         924 non-null    object
 3   url_video          917 non-null    object
 4   data_visualizacao  924 non-null    object
dtypes: object(5)
memory usage: 36.2+ KB


In [39]:
visualizao_youtube.head()


,canal,url_canal,nome_video,url_video,data_visualizacao
0,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,EDUARDO BOLSONARO E ANDRÉ MARINHO TRETARAM FEI...,https://www.youtube.com/watch?v=PD9Lk1YleWk,25 de jul. de 2021 17:55:05 BRT
1,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,GABRIEL MONTEIRO NO PÂNICO - MELHORES MOMENTOS...,https://www.youtube.com/watch?v=fOB0GYmAz7U,25 de jul. de 2021 17:55:00 BRT
2,Gabriel Monteiro,https://www.youtube.com/channel/UCYCMK_puWK2V5...,"VEIO ME EXPULSAR DO HOSPITAL, MAS FOI FLAGRADO...",https://www.youtube.com/watch?v=RpfoCFRGHbo,25 de jul. de 2021 17:54:57 BRT
3,Programação Dinâmica,https://www.youtube.com/channel/UC70mr11REaCqg...,RASPAGEM de DADOS com PYTHON usando BeautifulS...,https://www.youtube.com/watch?v=kqvWOcPog4s,25 de jul. de 2021 02:09:53 BRT
4,Jonny Kalambay,https://www.youtube.com/channel/UCUm0chMKj4MA7...,Como Utilizar la API de SpotifyJonny Kalambay2...,https://www.youtube.com/watch?v=PfL_v6A4fsI,23 de jul. de 2021 22:36:44 BRT


In [43]:
  def limpa_date(string):
    
    string = list(string)
    
    string =  ''.join(string).strip()
    string = string.split(' ')
   
    for i,valor in enumerate(string):
        if valor =='de' or valor == 'BRT':
            string.pop(i)     
        if i == 2:
            
            for chave in meses.keys():
                if meses[chave] == string[1].lower():
                    string[1] = chave
    string.insert(1,'/')
    string.insert(3,'/')
    string = ' '.join(str(v) for v in string) 
    return string



In [45]:
#Aplica funcao para aumentar letra do mes
visualizao_youtube['data_visualizacao'] = visualizao_youtube['data_visualizacao'].apply(limpa_date)

In [46]:
visualizao_youtube.head()

,canal,url_canal,nome_video,url_video,data_visualizacao
0,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,EDUARDO BOLSONARO E ANDRÉ MARINHO TRETARAM FEI...,https://www.youtube.com/watch?v=PD9Lk1YleWk,25 / 07 / 2021 17:55:05
1,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,GABRIEL MONTEIRO NO PÂNICO - MELHORES MOMENTOS...,https://www.youtube.com/watch?v=fOB0GYmAz7U,25 / 07 / 2021 17:55:00
2,Gabriel Monteiro,https://www.youtube.com/channel/UCYCMK_puWK2V5...,"VEIO ME EXPULSAR DO HOSPITAL, MAS FOI FLAGRADO...",https://www.youtube.com/watch?v=RpfoCFRGHbo,25 / 07 / 2021 17:54:57
3,Programação Dinâmica,https://www.youtube.com/channel/UC70mr11REaCqg...,RASPAGEM de DADOS com PYTHON usando BeautifulS...,https://www.youtube.com/watch?v=kqvWOcPog4s,25 / 07 / 2021 02:09:53
4,Jonny Kalambay,https://www.youtube.com/channel/UCUm0chMKj4MA7...,Como Utilizar la API de SpotifyJonny Kalambay2...,https://www.youtube.com/watch?v=PfL_v6A4fsI,23 / 07 / 2021 22:36:44


In [48]:
visualizao_youtube['data_visualizacao'] = pd.to_datetime(visualizao_youtube['data_visualizacao'],format  = '%d / %m / %Y %H:%M:%S')

In [49]:
visualizao_youtube.head()

,canal,url_canal,nome_video,url_video,data_visualizacao
0,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,EDUARDO BOLSONARO E ANDRÉ MARINHO TRETARAM FEI...,https://www.youtube.com/watch?v=PD9Lk1YleWk,2021-07-25 17:55:05
1,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,GABRIEL MONTEIRO NO PÂNICO - MELHORES MOMENTOS...,https://www.youtube.com/watch?v=fOB0GYmAz7U,2021-07-25 17:55:00
2,Gabriel Monteiro,https://www.youtube.com/channel/UCYCMK_puWK2V5...,"VEIO ME EXPULSAR DO HOSPITAL, MAS FOI FLAGRADO...",https://www.youtube.com/watch?v=RpfoCFRGHbo,2021-07-25 17:54:57
3,Programação Dinâmica,https://www.youtube.com/channel/UC70mr11REaCqg...,RASPAGEM de DADOS com PYTHON usando BeautifulS...,https://www.youtube.com/watch?v=kqvWOcPog4s,2021-07-25 02:09:53
4,Jonny Kalambay,https://www.youtube.com/channel/UCUm0chMKj4MA7...,Como Utilizar la API de SpotifyJonny Kalambay2...,https://www.youtube.com/watch?v=PfL_v6A4fsI,2021-07-23 22:36:44


In [50]:
#Excluir valores nulos de registro
#de visualização, pois eles são na maioria videos ou 
#canais que foi removido ou é privado,não conta para mim, pois não mostra nome video e nome do canal.
youtube_dado_limpo = visualizao_youtube.dropna()

In [51]:
youtube_dado_limpo.head()

,canal,url_canal,nome_video,url_video,data_visualizacao
0,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,EDUARDO BOLSONARO E ANDRÉ MARINHO TRETARAM FEI...,https://www.youtube.com/watch?v=PD9Lk1YleWk,2021-07-25 17:55:05
1,Pânico Retrô,https://www.youtube.com/channel/UC2IEgRZcIuemA...,GABRIEL MONTEIRO NO PÂNICO - MELHORES MOMENTOS...,https://www.youtube.com/watch?v=fOB0GYmAz7U,2021-07-25 17:55:00
2,Gabriel Monteiro,https://www.youtube.com/channel/UCYCMK_puWK2V5...,"VEIO ME EXPULSAR DO HOSPITAL, MAS FOI FLAGRADO...",https://www.youtube.com/watch?v=RpfoCFRGHbo,2021-07-25 17:54:57
3,Programação Dinâmica,https://www.youtube.com/channel/UC70mr11REaCqg...,RASPAGEM de DADOS com PYTHON usando BeautifulS...,https://www.youtube.com/watch?v=kqvWOcPog4s,2021-07-25 02:09:53
4,Jonny Kalambay,https://www.youtube.com/channel/UCUm0chMKj4MA7...,Como Utilizar la API de SpotifyJonny Kalambay2...,https://www.youtube.com/watch?v=PfL_v6A4fsI,2021-07-23 22:36:44


In [ ]:
#salavando dados como csv para usar no tableau
youtube_dado_limpo.to_csv('/home/mateus/Downloads/youtube/',index = False)